In [18]:
from docplex.mp.model import Model

In [19]:
m = Model(name='Cost Minimization')

# Variables

In [20]:
x = []
for i in range(1, 13):
    x.append(m.continuous_var(name=f'X {i}'))

In [21]:
# standards
manufacturing_time_per_unit = 1/3
cost_of_hour = 12
storage_cost_per_unit = 3

# these are monthly
cost_of_raw_materials_per_unit = [11, 10, 13, 9, 8, 7,
                                  10, 12, 12, 10, 9]
demand = [150, 200, 100, 300, 200,
          400, 300, 250, 150, 200, 300, 350]
avalaible_hours = [250, 250, 200, 150, 200, 200,
                   150, 200, 250, 150, 150, 200]
stored = 100

# Constraints

In [22]:
cost_sum = 0
for i in range(0, 12):
    m.add_constraint(manufacturing_time_per_unit*x[i]
                     <= avalaible_hours[i])
    m.add_constraint(x[i] + stored >= demand[i])
    stored = x[i] + stored - demand[i]
    cost_sum += manufacturing_time_per_unit*x[i]+stored*storage_cost_per_unit

# Goal

In [23]:
m.minimize(cost_sum)

# Couldn't get Docplex to work, tried with Pulp instead

# ---------------------------------------------------------------------------------------

In [1]:
from pulp import LpMinimize, LpProblem, LpStatus, lpSum, LpVariable

In [2]:
model = LpProblem(name="CostMinimization", sense=LpMinimize)
model_opt = LpProblem(name="CostMinimizationOpt", sense=LpMinimize)

# Variables

In [3]:
x = [LpVariable(name=f"x{i}", lowBound=0) for i in range(0, 12)]

In [4]:
# standards
manufacturing_time_per_unit = 1/3
cost_of_hour = 12
storage_cost_per_unit = 3

# these are monthly
cost_of_raw_materials_per_unit = [11, 10, 13, 9, 8, 7,
                                  10, 12, 12, 10, 9]
demand = [150, 200, 100, 300, 200,
          400, 300, 250, 150, 200, 300, 350]
avalaible_hours = [250, 250, 200, 150, 200, 200,
                   150, 200, 250, 150, 150, 200]

# Constraints

In [5]:
cost_sum = 0
stored = [100]

for i in range(1, 13):
    cost_constraint = manufacturing_time_per_unit*x[i-1] <= avalaible_hours[i-1]
    model += cost_constraint
    model_opt += cost_constraint
    demand_constraint = x[i-1] + stored >= demand[i-1]
    model += demand_constraint
    model_opt += demand_constraint
    stored.append(x[i-1] + stored - demand[i-1])
    cost_sum += manufacturing_time_per_unit*x[i-1]+stored[i-1]*storage_cost_per_unit
    storage_constraint = x[i-1] != 0
    constraint_opt = if x[i-1] > 0: cost_sum+=1000

SyntaxError: invalid syntax (2996304916.py, line 14)

In [33]:
model += cost_sum

In [34]:
model.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/m8/41547t_d4vbb207z7w8gbgyr0000gn/T/8860ad7471db404f92c6c952c879e191-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/m8/41547t_d4vbb207z7w8gbgyr0000gn/T/8860ad7471db404f92c6c952c879e191-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 29 COLUMNS
At line 132 RHS
At line 157 BOUNDS
At line 158 ENDATA
Problem MODEL has 24 rows, 12 columns and 90 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 11 (-13) rows, 12 (0) columns and 77 (-13) elements
0  Obj 153616.67 Primal inf 1012.7441 (11)
10  Obj 447566.67
Optimal - objective value 447566.67
After Postsolve, objective 447566.67, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 447566.6667 - 10 iterations time

1

In [35]:
model.objective.value()

866.6666666667086

In [36]:
for i in range(0, 12):
    print(x[i].value())

50.0
100.0
0.0
200.0
100.0
300.0
200.0
150.0
50.0
100.0
200.0
250.0


In [37]:
model

CostMinimization:
MINIMIZE
3072.3333333333335*x0 + 1536.3333333333335*x1 + 3.3333333333333335*x10 + 0.3333333333333333*x11 + 768.3333333333334*x2 + 384.33333333333337*x3 + 192.33333333333334*x4 + 96.33333333333334*x5 + 48.333333333333336*x6 + 24.333333333333336*x7 + 12.333333333333334*x8 + 6.333333333333334*x9 + -446700.0
SUBJECT TO
_C1: 0.333333333333 x0 <= 250

_C2: x0 >= 50

_C3: 0.333333333333 x1 <= 250

_C4: x0 + x1 >= 150

_C5: 0.333333333333 x2 <= 200

_C6: 2 x0 + x1 + x2 >= 200

_C7: 0.333333333333 x3 <= 150

_C8: 4 x0 + 2 x1 + x2 + x3 >= 600

_C9: 0.333333333333 x4 <= 200

_C10: 8 x0 + 4 x1 + 2 x2 + x3 + x4 >= 1100

_C11: 0.333333333333 x5 <= 200

_C12: 16 x0 + 8 x1 + 4 x2 + 2 x3 + x4 + x5 >= 2400

_C13: 0.333333333333 x6 <= 150

_C14: 32 x0 + 16 x1 + 8 x2 + 4 x3 + 2 x4 + x5 + x6 >= 4700

_C15: 0.333333333333 x7 <= 200

_C16: 64 x0 + 32 x1 + 16 x2 + 8 x3 + 4 x4 + 2 x5 + x6 + x7 >= 9350

_C17: 0.333333333333 x8 <= 250

_C18: 128 x0 + 64 x1 + 32 x2 + 16 x3 + 8 x4 + 4 x5 + 2 x6 +